In [ ]:
# путь до данных на компьютере
path = '/content/drive/MyDrive/files_and_description/train_data'

In [ ]:
# для работающих в Google Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

## Чтение в 222 признака

In [ ]:
dataset_paths = sorted([os.path.join(path, filename) for filename in os.listdir(path)
                            if filename.startswith('train')])
print(dataset_paths)

['/content/drive/MyDrive/files_and_description/train_data/train_data_0.pq', '/content/drive/MyDrive/files_and_description/train_data/train_data_1.pq', '/content/drive/MyDrive/files_and_description/train_data/train_data_10.pq', '/content/drive/MyDrive/files_and_description/train_data/train_data_11.pq', '/content/drive/MyDrive/files_and_description/train_data/train_data_2.pq', '/content/drive/MyDrive/files_and_description/train_data/train_data_3.pq', '/content/drive/MyDrive/files_and_description/train_data/train_data_4.pq', '/content/drive/MyDrive/files_and_description/train_data/train_data_5.pq', '/content/drive/MyDrive/files_and_description/train_data/train_data_6.pq', '/content/drive/MyDrive/files_and_description/train_data/train_data_7.pq', '/content/drive/MyDrive/files_and_description/train_data/train_data_8.pq', '/content/drive/MyDrive/files_and_description/train_data/train_data_9.pq']


Так как признаки бинаризованы (по сути, категориальные несравнимые признаки) и имеют малое количество уникальных значений, используем OHE-кодирование. При этом: удалим 4 признака про сроки сбора информации и к признакам, имеющих 2 уникальных значения, ohe-кодирование применять не будем (это признаки - флаги).

In [ ]:
# Сделаем эти списки постоянными, чтобы не зависеть от вариативности данных в разных датасетах
bi_cols = ['is_zero_loans3060', 'is_zero_loans6090', 'is_zero_loans90', 'is_zero_util', 'is_zero_over2limit', 'is_zero_maxover2limit', 'pclose_flag', 'fclose_flag']
cat_cols = ['pre_pterm', 'pre_fterm', 'pre_loans_credit_limit', 'pre_loans_next_pay_summ', 'pre_loans_outstanding', 'pre_loans_max_overdue_sum', 'pre_loans_credit_cost_rate', 'enc_paym_0', 'enc_paym_1', 'enc_paym_2', 'enc_paym_3', 'enc_paym_4', 'enc_paym_5', 'enc_paym_6', 'enc_paym_7', 'enc_paym_8', 'enc_paym_9', 'enc_paym_10', 'enc_paym_11', 'enc_paym_12', 'enc_paym_13', 'enc_paym_14', 'enc_paym_15', 'enc_paym_16', 'enc_paym_17', 'enc_paym_18', 'enc_paym_19', 'enc_paym_20', 'enc_paym_21', 'enc_paym_22', 'enc_paym_23', 'enc_paym_24', 'enc_loans_account_holder_type', 'enc_loans_credit_status', 'enc_loans_credit_type', 'enc_loans_account_cur']

In [ ]:
def bi_col_trans(data, cols, id='id', rn='rn'):
    trans_data = data.groupby(id)[rn].max().reset_index(drop=False)
    trans_data = pd.merge(trans_data, data.groupby(id)[[*cols]].sum().reset_index(drop=False), left_on=id, right_on=id)
    return trans_data

In [ ]:
def ohe_trans(data, col):
    ohe = OneHotEncoder(sparse=False)
    df = pd.DataFrame(data[col], columns=[col])
    ohe.fit(df)
    ohe_categorical = ohe.transform(df)
    data[ohe.get_feature_names_out()] = ohe_categorical.astype('int8')
    data = data.drop(col, axis=1)
    data = data.groupby('id')[ohe.get_feature_names_out()].sum().reset_index(drop=False)
    return data

In [ ]:
step = 0
for file_path in dataset_paths:

    df = pd.read_parquet(file_path)
    cols_for_drop =['pre_since_opened', 'pre_since_confirmed', 'pre_till_pclose', 'pre_till_fclose', 'pre_loans_total_overdue', 'pre_loans5', 'pre_loans530',
                    'is_zero_loans5', 'is_zero_loans530', 'pre_util', 'pre_over2limit', 'pre_maxover2limit', 'pre_loans3060', 'pre_loans6090', 'pre_loans90']
    for col in cols_for_drop:
        df = df.drop(col, axis=1)
    df[bi_cols] = df[bi_cols].astype('int8')
    df[cat_cols] = df[cat_cols].astype('int8')

    trans_data = bi_col_trans(df, bi_cols)

    for col in cat_cols:
        tmp_data = ohe_trans(df[['id', col]], col)
        trans_data = pd.merge(trans_data, tmp_data, left_on='id', right_on='id')
        del tmp_data

    block_as_str = str(step)
    if len(block_as_str) == 1:
        block_as_str = '00' + block_as_str
    else:
        block_as_str = '0' + block_as_str
    trans_data.to_parquet(os.path.join(path, f'min_processed_file_{block_as_str}.parquet'))
    step += 1
    del df
    del trans_data

Склейка обработанных файлов

In [ ]:
prep_paths = sorted([os.path.join(path, filename) for filename in os.listdir(path)
                            if filename.startswith('min_processed')])
print(prep_paths)

['/content/drive/MyDrive/files_and_description/train_data/min_processed_file_000.parquet', '/content/drive/MyDrive/files_and_description/train_data/min_processed_file_001.parquet', '/content/drive/MyDrive/files_and_description/train_data/min_processed_file_002.parquet', '/content/drive/MyDrive/files_and_description/train_data/min_processed_file_003.parquet', '/content/drive/MyDrive/files_and_description/train_data/min_processed_file_004.parquet', '/content/drive/MyDrive/files_and_description/train_data/min_processed_file_005.parquet', '/content/drive/MyDrive/files_and_description/train_data/min_processed_file_006.parquet', '/content/drive/MyDrive/files_and_description/train_data/min_processed_file_007.parquet', '/content/drive/MyDrive/files_and_description/train_data/min_processed_file_008.parquet', '/content/drive/MyDrive/files_and_description/train_data/min_processed_file_009.parquet', '/content/drive/MyDrive/files_and_description/train_data/min_processed_file_010.parquet', '/content

In [ ]:
result = pd.read_parquet(prep_paths[0])
for path in prep_paths[1:]:
    df = pd.read_parquet(path)
    result = pd.concat([result, df])
    result = result.fillna(0)
    del df


In [ ]:
result.shape

(3000000, 221)

In [ ]:
result = result.sort_values(by='id')

In [ ]:
result.head()

,id,rn,is_zero_loans3060,is_zero_loans6090,is_zero_loans90,is_zero_util,is_zero_over2limit,is_zero_maxover2limit,pclose_flag,fclose_flag,...,enc_loans_credit_type_3,enc_loans_credit_type_4,enc_loans_credit_type_5,enc_loans_account_cur_0,enc_loans_account_cur_1,enc_loans_account_cur_2,enc_loans_account_cur_3,enc_loans_credit_type_6,enc_loans_credit_type_7,pre_loans_max_overdue_sum_0
0,0,10,10,10,10,6,9,9,1,2,...,2,7,0,0,10,0,0,0.0,0.0,0.0
1,1,14,12,12,11,10,12,11,1,2,...,3,8,0,0,14,0,0,0.0,0.0,0.0
2,2,3,2,2,3,1,3,2,2,2,...,2,1,0,0,3,0,0,0.0,0.0,0.0
3,3,15,15,15,15,8,14,14,5,6,...,4,9,1,0,15,0,0,0.0,0.0,0.0
4,4,1,1,1,1,1,1,1,1,1,...,1,0,0,0,1,0,0,0.0,0.0,0.0


In [ ]:
path = '/content/drive/MyDrive/files_and_description/train_data'
result.to_parquet(os.path.join(path, f'/min_fin_proc.parquet'))

In [ ]:
result.to_parquet('/content/drive/MyDrive/files_and_description/train_data/min_fin_proc.parquet')

In [ ]:
df = pd.read_parquet(path + '/min_fin_proc.parquet')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/files_and_description/train_data/min_fin_proc.parquet'

In [ ]:
df = result.copy()

In [ ]:
df.head()

,id,rn,is_zero_loans3060,is_zero_loans6090,is_zero_loans90,is_zero_util,is_zero_over2limit,is_zero_maxover2limit,pclose_flag,fclose_flag,...,enc_loans_credit_type_3,enc_loans_credit_type_4,enc_loans_credit_type_5,enc_loans_account_cur_0,enc_loans_account_cur_1,enc_loans_account_cur_2,enc_loans_account_cur_3,enc_loans_credit_type_6,enc_loans_credit_type_7,pre_loans_max_overdue_sum_0
0,0,10,10,10,10,6,9,9,1,2,...,2,7,0,0,10,0,0,0.0,0.0,0.0
1,1,14,12,12,11,10,12,11,1,2,...,3,8,0,0,14,0,0,0.0,0.0,0.0
2,2,3,2,2,3,1,3,2,2,2,...,2,1,0,0,3,0,0,0.0,0.0,0.0
3,3,15,15,15,15,8,14,14,5,6,...,4,9,1,0,15,0,0,0.0,0.0,0.0
4,4,1,1,1,1,1,1,1,1,1,...,1,0,0,0,1,0,0,0.0,0.0,0.0


In [ ]:
targets = pd.read_csv('/content/drive/MyDrive/files_and_description/train_target.csv')
targets.head()

,id,flag
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [ ]:
fin_df = pd.merge(df, targets, left_on='id', right_on='id')

In [ ]:
fin_df.head()

,id,rn,is_zero_loans3060,is_zero_loans6090,is_zero_loans90,is_zero_util,is_zero_over2limit,is_zero_maxover2limit,pclose_flag,fclose_flag,...,enc_loans_credit_type_4,enc_loans_credit_type_5,enc_loans_account_cur_0,enc_loans_account_cur_1,enc_loans_account_cur_2,enc_loans_account_cur_3,enc_loans_credit_type_6,enc_loans_credit_type_7,pre_loans_max_overdue_sum_0,flag
0,0,10,10,10,10,6,9,9,1,2,...,7,0,0,10,0,0,0.0,0.0,0.0,0
1,1,14,12,12,11,10,12,11,1,2,...,8,0,0,14,0,0,0.0,0.0,0.0,0
2,2,3,2,2,3,1,3,2,2,2,...,1,0,0,3,0,0,0.0,0.0,0.0,0
3,3,15,15,15,15,8,14,14,5,6,...,9,1,0,15,0,0,0.0,0.0,0.0,0
4,4,1,1,1,1,1,1,1,1,1,...,0,0,0,1,0,0,0.0,0.0,0.0,0


In [ ]:
fin_df.isna().sum()

id                             0
rn                             0
is_zero_loans3060              0
is_zero_loans6090              0
is_zero_loans90                0
                              ..
enc_loans_account_cur_3        0
enc_loans_credit_type_6        0
enc_loans_credit_type_7        0
pre_loans_max_overdue_sum_0    0
flag                           0
Length: 222, dtype: int64

In [ ]:
X = fin_df.drop(['flag'], axis=1)
y = fin_df[['id', 'flag']]

x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [ ]:
x_train.to_parquet(os.path.join(path, f'min_x_train.parquet'))
x_test.to_parquet(os.path.join(path, f'min_x_test.parquet'))
y_train.to_parquet(os.path.join(path, f'min_y_train.parquet'))
y_test.to_parquet(os.path.join(path, f'min_y_test.parquet'))

In [ ]:
x_train.to_csv(os.path.join(path, f'x_train.csv'), index=False)
x_test.to_csv(os.path.join(path, f'x_test.csv'), index=False)
y_train.to_csv(os.path.join(path, f'y_train.csv'), index=False)
y_test.to_csv(os.path.join(path, f'y_test.csv'), index=False)

In [ ]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2100000 entries, 2870337 to 2219110
Columns: 338 entries, id to pre_loans530_17
dtypes: float64(25), int64(2), int8(311)
memory usage: 1.0 GB


## Downsampling in 222 feat

In [ ]:
# путь до данных на компьютере
path = '/content/drive/MyDrive/files_and_description/train_data'

In [ ]:
# для работающих в Google Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
from sklearn.utils import resample
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_parquet(path + '/min_fin_proc.parquet')

In [ ]:
targets = pd.read_csv('/content/drive/MyDrive/files_and_description/train_target.csv')
targets.head()

,id,flag
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [ ]:
fin_df = pd.merge(df, targets, left_on='id', right_on='id')

In [ ]:
fin_df.head()

,id,rn,is_zero_loans3060,is_zero_loans6090,is_zero_loans90,is_zero_util,is_zero_over2limit,is_zero_maxover2limit,pclose_flag,fclose_flag,...,enc_loans_credit_type_4,enc_loans_credit_type_5,enc_loans_account_cur_0,enc_loans_account_cur_1,enc_loans_account_cur_2,enc_loans_account_cur_3,enc_loans_credit_type_6,enc_loans_credit_type_7,pre_loans_max_overdue_sum_0,flag
0,0,10,10,10,10,6,9,9,1,2,...,7,0,0,10,0,0,0.0,0.0,0.0,0
1,1,14,12,12,11,10,12,11,1,2,...,8,0,0,14,0,0,0.0,0.0,0.0,0
2,2,3,2,2,3,1,3,2,2,2,...,1,0,0,3,0,0,0.0,0.0,0.0,0
3,3,15,15,15,15,8,14,14,5,6,...,9,1,0,15,0,0,0.0,0.0,0.0,0
4,4,1,1,1,1,1,1,1,1,1,...,0,0,0,1,0,0,0.0,0.0,0.0,0


In [ ]:
df_min = fin_df[fin_df['flag'] == 1]
df_maj = fin_df[fin_df['flag'] == 0]

df_maj_downsample = resample(df_maj, replace=False, n_samples=len(df_min), random_state=1)

print(df_maj_downsample.shape)
print(df_min.shape)

(106442, 222)
(106442, 222)


In [ ]:
df_2 = pd.concat([df_maj_downsample, df_min], ignore_index=True).sample(frac=1.)

In [ ]:
df_2.head()

,id,rn,is_zero_loans3060,is_zero_loans6090,is_zero_loans90,is_zero_util,is_zero_over2limit,is_zero_maxover2limit,pclose_flag,fclose_flag,...,enc_loans_credit_type_4,enc_loans_credit_type_5,enc_loans_account_cur_0,enc_loans_account_cur_1,enc_loans_account_cur_2,enc_loans_account_cur_3,enc_loans_credit_type_6,enc_loans_credit_type_7,pre_loans_max_overdue_sum_0,flag
13225,1073865,4,4,4,4,2,4,4,0,1,...,1,0,0,2,2,0,0.0,0.0,0.0,0
39136,2760255,2,2,2,2,1,2,2,0,0,...,2,0,0,2,0,0,0.0,0.0,0.0,0
41580,1880221,9,5,7,6,5,8,7,1,3,...,4,0,0,9,0,0,0.0,0.0,0.0,0
203410,2703565,2,2,2,2,0,2,2,1,1,...,1,0,0,2,0,0,0.0,0.0,0.0,1
153699,1339337,16,16,16,16,9,15,15,1,3,...,10,0,0,16,0,0,0.0,1.0,0.0,1


In [ ]:
X = df_2.drop(['flag'], axis=1)
y = df_2[['id', 'flag']]

x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [ ]:
x_train.to_parquet(os.path.join(path, f'ds_min_x_train.parquet'))
x_test.to_parquet(os.path.join(path, f'ds_min_x_test.parquet'))
y_train.to_parquet(os.path.join(path, f'ds_min_y_train.parquet'))
y_test.to_parquet(os.path.join(path, f'ds_min_y_test.parquet'))

## Tecnical

In [ ]:
df = pd.read_parquet(dataset_paths[0])

In [ ]:
df.head()

In [ ]:
for col in ['pre_since_opened', 'pre_since_confirmed', 'pre_till_pclose', 'pre_till_fclose', 'pre_loans_total_overdue']:
    df = df.drop(col, axis=1)

In [ ]:
df[bi_cols] = df[bi_cols].astype('int8')
df[cat_cols] = df[cat_cols].astype('int8')

In [ ]:
df.info()

In [ ]:
def bi_col_trans(data, cols, id='id', rn='rn'):
    trans_data = data.groupby(id)[rn].max().reset_index(drop=False)
    trans_data = pd.merge(trans_data, data.groupby(id)[[*cols]].sum().reset_index(drop=False), left_on=id, right_on=id)
    return trans_data

In [ ]:
trans_data = bi_col_trans(df, bi_cols)

In [ ]:
trans_data.info()

In [ ]:
def ohe_trans(data, col):
    ohe = OneHotEncoder(sparse=False)
    df = pd.DataFrame(data[col], columns=[col])
    ohe.fit(df)
    ohe_categorical = ohe.transform(df)
    data[ohe.get_feature_names_out()] = ohe_categorical.astype('int8')
    data = data.drop(col, axis=1)
    data = data.groupby('id')[ohe.get_feature_names_out()].sum().reset_index(drop=False)
    return data

In [ ]:
for col in cat_cols:
    tmp_data = ohe_trans(df[['id', col]], col)
    trans_data = pd.merge(trans_data, tmp_data, left_on='id', right_on='id')
    del tmp_data

In [ ]:
trans_data.info()

In [ ]:
trans_data.isna().sum()

In [ ]:
cols = df.columns.to_list()

for col in cols:
    print(f'{col}, {df[col].nunique()}')

id, 250000
rn, 51
pre_since_opened, 20
pre_since_confirmed, 18
pre_pterm, 18
pre_fterm, 17
pre_till_pclose, 17
pre_till_fclose, 16
pre_loans_credit_limit, 20
pre_loans_next_pay_summ, 7
pre_loans_outstanding, 5
pre_loans_total_overdue, 1
pre_loans_max_overdue_sum, 3
pre_loans_credit_cost_rate, 14
pre_loans5, 8
pre_loans530, 15
pre_loans3060, 5
pre_loans6090, 3
pre_loans90, 4
is_zero_loans5, 2
is_zero_loans530, 2
is_zero_loans3060, 2
is_zero_loans6090, 2
is_zero_loans90, 2
pre_util, 20
pre_over2limit, 20
pre_maxover2limit, 20
is_zero_util, 2
is_zero_over2limit, 2
is_zero_maxover2limit, 2
enc_paym_0, 4
enc_paym_1, 4
enc_paym_2, 4
enc_paym_3, 4
enc_paym_4, 4
enc_paym_5, 4
enc_paym_6, 4
enc_paym_7, 4
enc_paym_8, 4
enc_paym_9, 4
enc_paym_10, 4
enc_paym_11, 4
enc_paym_12, 4
enc_paym_13, 4
enc_paym_14, 4
enc_paym_15, 4
enc_paym_16, 4
enc_paym_17, 4
enc_paym_18, 4
enc_paym_19, 4
enc_paym_20, 4
enc_paym_21, 4
enc_paym_22, 4
enc_paym_23, 4
enc_paym_24, 4
enc_loans_account_holder_type, 7
enc_loans

In [ ]:
bi_cols = []
cat_cols = []
for col in cols:
    if df[col].nunique() > 2:
        cat_cols.append(col)
    else:
        bi_cols.append(col)

print(bi_cols)
print(cat_cols)

In [ ]:
trans_data = df[['id', 'rn', *bi_cols]]

In [ ]:
trans_data = pd.merge(trans_data, tmp_data, left_on='id', right_on='id')
trans_data

Так как признаки бинаризованы (по сути, категориальные несравнимые признаки) и имеют малое количество уникальных значений, используем OHE-кодирование.